# Paper Review : Doubly Stochastic Variational Inference for Deep Gaussian Processes

Deep Gaussian Process (DGP), a hierarchical composition of Gaussian Processes(GP), can overcome the limitations of standard (single-layer) GP while maintaining the benefits of GP<sup>[1]</sup>.

$\require{cancel}$

## Standard (Single-layer) Gaussian Processes  

>Consider inferring a stochastic function $f:\mathbb{R}^{D}\to\mathbb{R}$ , given:
- A likelihood $p(y|f)$, models how observations $y$ relate to (latent) function values $f$.
- A dataset of $N$ observed outputs $\mathbf{y}=(y_{1},\dots,y_{N})^{\top}$ at input (design) locations $\mathbf X=(\mathbf x_{1},\dots,\mathbf x_{N})^{\top}$. 

> Place a GP prior on function $f$, meaning that any finite collection of function values follows a joint Gaussian distribution, with:
- A mean function $m:\mathbb{R}^{D}\rightarrow\mathbb{R}$.
- A covariance function (kernel) $k:\mathbb{R}^{D}\times\mathbb{R}^{D}\overset{.}{\to}\mathbb{R}$ encodes similarities.  

> **Inducing points** for sparse approximation:

1. Define an additional set of $M$ inducing locations $\mathbf{Z}=(\mathbf{z}_{1}, \cdots,\mathbf{z}_{M})^{\top}$.

2. Write:
- $\mathbf{f}=f(\mathbf{X})$, $N$-dimensional vector of function values at the observed locations.
- $\mathbf{u}=f(\mathbf{Z})$, $M$-dimensional vector of function values at inducing locations.

By the GP prior, the joint distribution over $(\mathbf{f},\mathbf{u})$ is multivariate Gaussian: 
$$
\begin{bmatrix} \mathbf{f} \\ \mathbf{u} \end{bmatrix} \sim \mathcal{N} \left( \begin{bmatrix} m(\mathbf{X}) \\ m(\mathbf{Z}) \end{bmatrix}, \begin{bmatrix} K_{\mathbf{XX}} & K_{\mathbf{XZ}} \\ K_{\mathbf{ZX}} & K_{\mathbf{ZZ}} \end{bmatrix} \right) \tag{1}
$$
- $K_{\mathbf{XX}}$: $N \times N$ matrix with entries $[K_{\mathbf{XX}}]_{ij} = k(\mathbf{x}_i, \mathbf{x}_j)$.
- $K_{\mathbf{ZZ}}$: $M \times M$ matrix with entries $[K_{\mathbf{ZZ}}]_{ij} = k(\mathbf{z}_i, \mathbf{z}_j)$.
- $K_{\mathbf{XZ}}$ and $K_{\mathbf{ZX}} = K_{\mathbf{XZ}}^{\top}$ link the observed and inducing locations.

The joint density of $\mathbf{y},\mathbf{f}$ and $\mathbf{u}$ is given by:  

$$
\begin{aligned}
p(\mathbf{y},\mathbf{f},\mathbf{u})&= p(\mathbf{f}, \mathbf{u}) p(\mathbf{y}|\mathbf{f}, \cancel{\mathbf{u}}) \\
&= p(\mathbf{f}|\mathbf{u}) p(\mathbf{u}) \prod_{i=1}^{N}p(y_{i}|f_{i}) \\
&=\underbrace{p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})p(\mathbf{u};\mathbf{Z})}_{\mathrm{GP~prior}}\underbrace{\prod_{i=1}^{N}p(y_{i}|f_{i})}_{\mathrm{likelihood}}
\end{aligned} \tag{2}
$$  

**Notice** that $p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})$ indicates that the input locations for $\mathbf{f}$ and $\mathbf{u}$ are $\mathbf{X}$ and $\mathbf{Z}$, respectively.

1. The prior $p(\mathbf{u}; \mathbf{Z}) = \mathcal{N}(\mathbf{u} | m(\mathbf{Z}), k(\mathbf{Z}, \mathbf{Z}))$
2. the conditional $p(\mathbf{f} | \mathbf{u}; \mathbf{X}, \mathbf{Z}) = \mathcal{N}(\mathbf{f} | \boldsymbol{\mu}, \mathbf{\Sigma})$, where for $i, j = 1, \ldots, N$:

$$[\boldsymbol{\mu}]_i = m(\mathbf{x}_i) + \boldsymbol{\alpha}(\mathbf{x}_i)^{\top} (\mathbf{u} - m(\mathbf{Z})) \tag{3}$$
$$[\mathbf{\Sigma}]_{ij} = k(\mathbf{x}_i, \mathbf{x}_j) - \boldsymbol{\alpha}(\mathbf{x}_i)^{\top} k(\mathbf{Z}, \mathbf{Z}) \boldsymbol{\alpha}(\mathbf{x}_j) \tag{4}$$

with $\boldsymbol{\alpha}(\mathbf{x}_{i})=k(\mathbf{Z},\mathbf{Z})^{-1}k(\mathbf{Z},\mathbf{x}_{i})$. Inference is possible in closed form when the likelihood $p(y|f)$ is Gaussian, but the time complexity is $O(N^3)$.

**Variational Inference for Sparse GP**

To handle **(a)** large datasets ($N \gg M$) **(b)** non-Gaussian likelihoods simultaneously, we need to seek a variational posterior.

**1.** Variational inference aims to approximate the true posterior distribution $p(\mathbf{f},\mathbf{u})$ with a simpler distribution $q(\mathbf{f},\mathbf{u})$, by minimizing the Kullback-Leibler divergence $\operatorname{KL}[q||p]$ between the variational posterior $q$ and the true posterior $p$ . Equivalently, this can be achieved by maximizing the a lower bound on the marginal likelihood (a.k.a. evidence):  

- Start from the log-marginal likelihood ("evidence") of the observations $\mathbf{y}$:

$$\log p(\mathbf{y}) = \log \int p(\mathbf{y},\mathbf{f},\mathbf{u}) d\mathbf{f} d\mathbf{u} = \log \int q(\mathbf{f},\mathbf{u}) \frac{p(\mathbf{y},\mathbf{f},\mathbf{u})}{q(\mathbf{f},\mathbf{u})} d\mathbf{f} d\mathbf{u}$$

- Apply Jensen's inequality with the expectation taken w.r.t $q(\mathbf{f},\mathbf{u})$ ($\log$ is concave):

$$$$

$$
\log p(\mathbf{y}) \geq \mathbb{E}_{q(\mathbf{f},\mathbf{u})}\left[\log\frac{p(\mathbf{y},\mathbf{f},\mathbf{u})}{q(\mathbf{f},\mathbf{u})}\right] = \mathcal{L} \tag{5}
$$  

where $p(\mathbf{y},\mathbf{f},\mathbf{u})$ is given in (2). 

**2.** Choose a variational posterior based on the work of Hensman et al. (2013), **_Gaussian process for Big Data_**:

$$
q(\mathbf{f},\mathbf{u})=p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})q(\mathbf{u}) \tag{6}
$$  

where $q(\mathbf{u})=\mathcal{N}(\mathbf{u}|\mathbf{m},\mathbf{S})$.

**3.** Since both terms in the variational posterior are Gaussian, marginalizing $\mathbf{u}$ yields:  

$$
q(\mathbf{f}|\mathbf{m},\mathbf{S};\mathbf{X},\mathbf{Z})=\int p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})q(\mathbf{u})d\mathbf{u}=\mathcal{N}(\mathbf{f}|\tilde{\boldsymbol{\mu}},\tilde{\boldsymbol{\Sigma}}) \tag{7}
$$  

$\tilde{\boldsymbol{\mu}}$ and $\tilde{\boldsymbol{\Sigma}}$ can be expressed as the mean and covariance functions of the inputs, similar to equations (3) and (4). This form is commonly used in Bayesian inference with Gaussian random fields (GRF):

$$
\mu_{\mathbf{m},\mathbf{Z}}(\mathbf{x}_{i})=m(\mathbf{x}_{i})+\boldsymbol{\alpha}(\mathbf{x}_{i})^{\top}(\mathbf{m}-m(\mathbf{Z})) \tag{8}
$$
$$
\Sigma_{\mathbf{S},\mathbf{Z}}(\mathbf{x}_{i},\mathbf{x}_{j})=k(\mathbf{x}_{i},\mathbf{x}_{j})-\boldsymbol{\alpha}(\mathbf{x}_{i})^{\top}(k(\mathbf{Z},\mathbf{Z})-\mathbf{S})\boldsymbol{\alpha}(\mathbf{x}_{j}) \tag{9}
$$  

**Remark**

The $f_{i}$ marginals of the variational posterior (7) depend only on the corresponding inputs $\mathbf{x}_{i}$ . Therefore, the i<sup>th</sup> marginal of $q(\mathbf{f}|\mathbf{m},\mathbf{S};\mathbf{X},\mathbf{Z})$ can be written as:  

$$
q(f_{i}|\mathbf{m},\mathbf{S};\mathbf{X},\mathbf{Z})=q(f_{i}|\mathbf{m},\mathbf{S};\mathbf{x}_{i},\mathbf{Z})=\mathcal{N}(f_{i}|\mu_{\mathbf{m},\mathbf{Z}}(\mathbf{x}_{i}),\Sigma_{\mathbf{S},\mathbf{Z}}(\mathbf{x}_{i},\mathbf{x}_{i})) \tag{10}
$$ 
<pre> <code>
 Observations:      y1         y*         yc
                    |          |          |
Gaussian field:     f1 -- o -- f* -- o -- fc
                    |     |    |     |    |
     Inputs:        x1    x2   x*   ...   xc
</code> </pre>

Using the variational posterior (6) the lower bound (5) simplifies considerably since:
- The conditionals $p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})$ inside the logarithm cancel
- The likelihood expectation requires only the variational marginals

Let's start with the evidence lower bound (ELBO) from (5):
$$\begin{aligned}\mathcal{L}&=\mathbb{E}_{q(\mathbf{f},\mathbf{u})}\left[\log\frac{p(\mathbf{y},\mathbf{f},\mathbf{u})}{q(\mathbf{f},\mathbf{u})}\right]\end{aligned}$$
From (6), we have:
$$q(\mathbf{f},\mathbf{u})=p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})q(\mathbf{u})$$ 
From (2), we know that:
$$p(\mathbf{y},\mathbf{f},\mathbf{u})=p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})p(\mathbf{u};\mathbf{Z})\prod_{i=1}^{N}p(y_{i}|f_{i})$$
So the ratio inside the logarithm is:
$$
\begin{aligned}
\frac{p(\mathbf{y}, \mathbf{f}, \mathbf{u})}{q(\mathbf{f}, \mathbf{u})} &= \frac{\cancel{p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})}p(\mathbf{u};\mathbf{Z})\prod_{i=1}^{N}p(y_{i}|f_{i})}{\cancel{p(\mathbf{f}|\mathbf{u};\mathbf{X},\mathbf{Z})}q(\mathbf{u})} \\
&= \frac{p(\mathbf{u};\mathbf{Z})\prod_{i=1}^{N}p(y_{i}|f_{i})}{q(\mathbf{u})}
\end{aligned}
$$

Substituting back into the ELBO, we obtain:
$$
\begin{aligned}
\mathcal{L}&=\mathbb{E}_{q(\mathbf{f},\mathbf{u})}\left[\log\frac{p(\mathbf{y},\mathbf{f},\mathbf{u})}{q(\mathbf{f},\mathbf{u})}\right] \\
&= \mathbb{E}_{q(\mathbf{f},\mathbf{u})}\left[\log\frac{p(\mathbf{u};\mathbf{Z})\prod_{i=1}^{N}p(y_{i}|f_{i})}{q(\mathbf{u})}\right] \\
&= \mathbb{E}_{q(\mathbf{u})}\left[\log\frac{p(\mathbf{u})}{q(\mathbf{u})}\right] + \mathbb{E}_{q(\mathbf{f})}\left[\log\prod_{i=1}^{N}p(y_{i}|f_{i})\right] \\
&= - \mathrm{KL}[q(\mathbf{u})||p(\mathbf{u})] + \sum_{i=1}^{N}\mathbb{E}_{q(\mathbf{f_i})}\left[\log p(y_{i}|f_{i})\right]
\end{aligned}
$$

Since the expectation of the likelihood only requires the marginal variational distributions for each $f_{i}$. From (10) we know that the marginals are given by:
$$q(f_{i}|\mathbf{m},\mathbf{S};\mathbf{x}_{i},\mathbf{Z})=\mathcal{N}(f_{i}|\mu_{\mathbf{m},\mathbf{Z}}(\mathbf{x}_{i}),\Sigma_{\mathbf{S},\mathbf{Z}}(\mathbf{x}_{i},\mathbf{x}_{i}))$$

Thus we arrive at the simplified ELBO:
$$\mathcal{L}=\sum_{i=1}^{N}\mathbb{E}_{q(f_{i}|\mathbf{m},\mathbf{S};\mathbf{x}_{i},\mathbf{Z})}[\log p(y_{i}|f_{i})]-\mathrm{KL}[q(\mathbf{u})||p(\mathbf{u})] \tag{11}$$
 

The final (univariate) expectation of the log-likelihood ($\color{red}{\mathbb{E}_{q(f_{i}|\mathbf{m},\mathbf{S};\mathbf{x}_{i},\mathbf{Z})}[\log p(y_{i}|f_{i})]}$) can be computed analytically in some cases:
- Gauss-Hermite quadrature, Hensman et al., 2015, **_Scalable Variational Gaussian Process Classification_**;
- Monte Carlo sampling, Bonilla et al., 2016 **_Generic Inference in Latent Gaussian Process Models_** and Gal et al., 2015 **_Latent Gaussian Processes for Distribution Estimation of Multivariate Categorical Data_**. 

The bound is a sum over the data, an unbiased estimator can be obtained through minibatch subsampling. This makes inference feasible on large datasets. A GP uses this inference method referred as a **sparse Gaussian Process (SGP)**.

The variational parameters $(\mathbf{Z},\mathbf{m}$ and $\mathbf{S})$ are optimized by maximizing the lower bound we just derived (11). This maximization is guaranteed to converge, as $\mathcal{L}$ is a valid lower bound to the marginal likelihood $p(\mathbf{y}|\mathbf{X})$. The model parameters (hyperparameters of the kernel and likelihood) can also be learned through the maximization this bound. However, caution is needed, as this approach may introduce bias—the bound is not uniformly tight across all hyperparameter settings.

For $D$-dimensional outputs $\mathbf{y}_{i}\in\mathbb{R}^{D}$, define $\mathbf{Y}$ as the matrix with $i$<sup>th</sup> row containing the $i$<sup>th</sup> observation $\mathbf{y}_{i}$. Similarly, define $\mathbf{F}$ and $\mathbf{U}$. If each output is an independent GP we have the GP prior $\begin{array}{r}{\prod_{d=1}^{D}p(\mathbf{F}_{d}|\mathbf{U}_{d};\mathbf{X},\mathbf{Z})p(\mathbf{U}_{d};\mathbf{Z})}\end{array}$.

## Deep Gaussian Processes

>A Deep Gaussian Process (DGP) (Damianou and Lawrence, 2013) defines a prior recursively on vector-valued stochastic functions $F^{1}, F^{2},\ldots,F^{L}$.

- Layer-wise GPs:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; At layer $l$, we have $D^{l}$ scalar functions $\{F_{d}^{l}\}_{d=1}^{D^{l}}$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Each $F_{d}^{l}$ is a GP whose inputs are noisy corruptions of the outputs from the previous layer, $F^{l-1}$.

- Conditional independence assumption:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Given the input $F^{l-1}$, the $D^{l}$ outputs at layer $l$ are conditionally independent.

- Example:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Pick M inducing input locations, $Z^{l-1} = \{z_m^{l-1} \in \mathbb{R}^{D^{l-1}}\}_{m=1}^M$, then collect the corresponding function values into:
$$
U^l = \begin{bmatrix}
f_1^l(z_1^{l-1}) & \cdots & f_{D^l}^l(z_1^{l-1}) \\
\vdots & \ddots & \vdots \\
f_1^l(z_M^{l-1}) & \cdots & f_{D^l}^l(z_M^{l-1})
\end{bmatrix} \in \mathbb{R}^{M \times D^l}
$$

>The noise between layers is assumed i.i.d. Gaussian. Most presentations of DGPs (see, e.g. Damianou and Lawrence, 2013; Bui et al., 2016) explicitly parameterize the noisy corruptions separately from the outputs of each GP.

The authors' method of inference does not require us to parameterize these variables separately. For notational convenience, we therefore absorb the noise into the kernel $k_{n o i s y}(\mathbf{x}_{i},\mathbf{x}_{j})=\bar{k}(\mathbf{x}_{i},\mathbf{x}_{j})+\sigma_{l}^{2}\delta_{i j}$ , where $\delta_{i j}$ is the Kronecker delta, and $\sigma_{l}^{2}$ is the noise variance between layers.

We still use inducing locations $\mathbf{Z}^{l-1}$ and corresponding inducing outputs $\mathbf{U}^{l}$ at each layer.

**Joint model and intractable inference**

With $\mathbf{F}^{0}=\mathbf{X}$, the full joint density as:

$$
p({\mathbf{Y}},\{{\mathbf{F}}^{l},{\mathbf{U}}^{l}\}_{l=1}^{L})=\underbrace{\prod_{i=1}^{N}p({\mathbf{y}}_{i}|{\mathbf{f}}_{i}^{L})}_{\mathrm{likelihood}}\underbrace{\prod_{l=1}^{L}p({\mathbf{F}}^{l}|{\mathbf{U}}^{l};{\mathbf{F}}^{l-1},{\mathbf{Z}}^{l-1})p({\mathbf{U}}^{l};{\mathbf{Z}}^{l-1})}_{\mathrm{DGP~prior}},
$$  

Inference in this model is intractable, so we rely on variational approximations.

**Variational schemes comparison**

The original DGP presentation (Damianou and Lawrence, 2013) uses a variational posterior that maintains the exact model conditioned on $\mathbf{U}^{l}$ , but further forces the inputs to each layer to be independent from the outputs of the previous layer. 

- The noisy corruptions are parameterized separately, and the variational distribution over these variables is a fully factorized Gaussian.

- This approach requires $2N(D^{1}+\cdot\cdot\cdot+D^{L-1})$ variational parameters but admits a tractable lower bound on the log marginal likelihood if the kernel is of a particular form.

- A further problem of this bound is that the density over the outputs is simply a single layer GP with independent Gaussian inputs. Since the posterior loses all the correlations between layers it cannot express the complexity of the full model and so is likely to underestimate the variance. In practice, we found that optimizing the objective in Damianou and Lawrence (2013) results in layers being ‘turned off’ (the signal to noise ratio tends to zero). 

In contrast, our posterior retains the full conditional structure of the true model. We sacrifice analytical tractability, but due to the sparse posterior within each layer we can sample the bound using univariate Gaussians.

## Reference<br>
<!-- apa -->
[1] Salimbeni, H., & Deisenroth, M. (2017). Doubly stochastic variational inference for deep Gaussian processes. Advances in neural information processing systems, 30.